In [523]:
# %pip install scikit-learn
# %pip install xgboost

import pandas as pd
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import plot_importance
import matplotlib.pyplot as plt

In [524]:
sales_file="../../Data/brand details last 1 year.xlsx"
sales_data=pd.read_excel(sales_file,sheet_name='Healthkart')
sales_data['SKU_grouped']=sales_data['SKU Name'].str.split(',',expand=True)[0]
sales_data

,Order Date & Time,Brand Name,SKU Name,Quantity,Item Value,Delivery Pincode,Payment Type,SKU_grouped
0,2023-10-28 07:34:26,HEALTHKART,"MuscleBlaze Limited Edition Shubman Shaker, B...",1,699.0,201306,PREPAID,MuscleBlaze Limited Edition Shubman Shaker
1,2023-10-28 07:34:26,HEALTHKART,"MuscleBlaze High Protein Muesli, 0.4 kg Dark...",1,399.0,201309,COD,MuscleBlaze High Protein Muesli
2,2023-10-28 07:34:27,HEALTHKART,"MuscleBlaze High Protein Gold Gainer, 2.2 lb ...",1,2129.0,201301,PREPAID,MuscleBlaze High Protein Gold Gainer
3,2023-10-28 07:34:27,HEALTHKART,HK Vitals ProteinUp Active Strength & Energy b...,1,799.0,201303,PREPAID,HK Vitals ProteinUp Active Strength & Energy b...
4,2023-10-28 07:34:28,HEALTHKART,"MuscleBlaze Biozyme Performance Whey, 4.4 lb ...",1,5499.0,201301,PREPAID,MuscleBlaze Biozyme Performance Whey
...,...,...,...,...,...,...,...,...
104421,2024-10-28 14:52:06,HEALTHKART,"Supermilk Height+Â for Boy/Girl 4 to 7 years, ...",1,1348.0,110092,PREPAID,Supermilk Height+Â for Boy/Girl 4 to 7 years
104422,2024-10-28 14:57:39,HEALTHKART,"HealthKart HK Vitals Skin Radiance Collagen, ...",1,1748.0,201304,COD,HealthKart HK Vitals Skin Radiance Collagen
104423,2024-10-28 15:06:14,healthkart,"MuscleBlaze T-Surge Black, 90 tablet(s) Unfl...",1,1399.0,121005,COD,MuscleBlaze T-Surge Black
104424,2024-10-28 15:10:41,healthkart,"HealthKart HK Vitals Skin Radiance Collagen, ...",1,1099.0,110081,COD,HealthKart HK Vitals Skin Radiance Collagen


In [525]:
sales_data['SKU Name']=sales_data['SKU Name'].str.split('_')
sales_data=sales_data.explode('SKU Name',ignore_index=True)
sales_data['SKU_grouped']=sales_data['SKU Name'].str.split(',',expand=True)[0]
sales_data['Order Date & Time']=pd.to_datetime(sales_data['Order Date & Time'])
sales_data['Date']=sales_data['Order Date & Time'].dt.date
sales_data_grouped=sales_data.groupby(['SKU_grouped','Date']).agg({'Quantity':'sum',"Item Value":"sum","Brand Name":"first"}).reset_index()
sales_data_grouped=sales_data_grouped.sort_values('Date') 
sales_data_grouped_select=sales_data.groupby(['SKU_grouped']).agg({'Quantity':'sum',"Date":'nunique'}).reset_index()

sales_data_grouped_select=sales_data_grouped_select.rename(columns={'Quantity': 'Total Sales Value',"Date":'Date Count'})
sales_data_grouped_select=sales_data_grouped_select.sort_values(by=['Date Count'],ascending=[False])
sales_data_grouped_select=sales_data_grouped_select.head(20)

sales_data_grouped_select=sales_data_grouped_select.merge(sales_data_grouped,on='SKU_grouped',how='left')
sales_data_grouped_select.to_clipboard()

In [526]:
sales_data_grouped_select=sales_data_grouped_select.sort_values('Date')
sales_data_grouped_select["SKU_grouped"].nunique()

20

In [527]:
# Define date range
start_date = datetime(2023, 1, 2)
end_date = datetime(2025, 11, 30)

# Generate daily dates
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Define Indian holidays including Dussehra and Black Friday
indian_holidays = { 
    '2023-01-01': 'New Year',
    '2023-01-26': 'Republic Day',
    '2023-08-15': 'Independence Day',
    '2023-10-24': 'Dussehra',
    '2023-10-02': 'Gandhi Jayanti',
    '2023-11-12': 'Diwali',
    '2023-11-24': 'Black Friday',
    '2023-12-25': 'Chrismas',
    '2024-01-01': 'New Year',
    '2024-01-26': 'Republic Day',
    '2024-08-15': 'Independence Day',
    '2024-10-13': 'Dussehra',
    '2024-10-02': 'Gandhi Jayanti',
    '2024-11-01': 'Diwali',
    '2024-11-29': 'Black Friday',
    '2024-12-25': 'Chrismas',
    '2025-01-01': 'New Year',
    '2025-01-26': 'Republic Day',
    '2025-08-15': 'Independence Day',
    '2025-10-02': 'Gandhi Jayanti',
    '2025-10-02': 'Gandhi Jayanti',
    '2025-10-21': 'Dussehra',
    '2025-10-20': 'Diwali',
    '2025-11-28': 'Black Friday', 
    '2025-12-25': 'Chrismas'
}

# Create a date_mapFrame
date_map = pd.DataFrame({'Date': dates})

# Add columns
date_map['Year'] = date_map['Date'].dt.year                       # Extract year
date_map['Month'] = date_map['Date'].dt.month  
date_map['Day'] = date_map['Date'].dt.day    
date_map['Week of Year'] = date_map['Date'].dt.isocalendar().week  # ISO week number
#date_map['Week_Sequence'] = date_map['Year'].astype(str) +"_"+date_map['Week of Year'].astype(str)
date_map['Month and Year'] = date_map['Year'].astype(str) +"_"+date_map['Month'].astype(str)
month_and_year = date_map['Month and Year'].unique()
date_map['Holiday Name'] = date_map['Date'].dt.strftime('%Y-%m-%d').map(indian_holidays)
# Add a column to mark whether the date is a holiday or weekend
#date_map['Is Weekend'] = np.where(date_map['Day of Week'].isin(['Saturday', 'Sunday']),1,0)
date_map['Is Holiday'] = date_map['Holiday Name'].notna().astype(int) 

date_map['Week_Group'] = (date_map.index // 7) + 1

#date_map=date_map.to_clipboard()




In [528]:

#Add 14 days before festival
#date_map['Date'] = pd.to_datetime(date_map['Date'])
date_map['is_14_days_before_holiday']=0
for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
    date_map.loc[
        (date_map['Date']>=date - pd.Timedelta(days=14)) & (date_map['Date']<date),
         'is_14_days_before_festival']=1 

#Add a week before festival
date_map['is_a_week_before_holiday']=0
for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
    date_map.loc[
        (date_map['Date']>=date - pd.Timedelta(days=14)) & (date_map['Date']<date -pd.Timedelta(days=7)),
         'is_a_week_before_holiday']=1 

#Add 7 days before festival
date_map['is_7_days_before_holiday']=0
for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
    date_map.loc[
        (date_map['Date']>=date-pd.Timedelta(days=7)) & (date_map['Date']<date),
         'is_7_days_before_festival']=1 
date_map.to_clipboard()    


In [529]:
#date_map['Week of Month'] = (date_map['Date'].dt.day-1) // 7 + 1
prev_hold=0
last_week_prevmonth=0
adj_factor1=0
final_df= pd.DataFrame()
for month in month_and_year:
    main_df=date_map[date_map['Month and Year']==month]
    num_days = len(main_df)
    num_week=(num_days-(7-prev_hold)) // 7+1
    if prev_hold==0:
        prev_hold_adj=7
        adj_factor=1
    else:
        prev_hold_adj= prev_hold  
        adj_factor=2
    if prev_hold<=3 and prev_hold>0 :
        Week_numtoimpute=1
        adj_factor=2
        adj_factor1=1
        #adj_factor=1
    elif prev_hold==0:
        Week_numtoimpute=max_week
        adj_factor=1
        adj_factor1=0
    else:    
        Week_numtoimpute=max_week
        adj_factor=1
        adj_factor1=1
    main_df['prevhold']=prev_hold   
    main_df['test']=main_df['Day']-(7-prev_hold_adj+1) -1
    main_df['test2']=7-prev_hold_adj+1 -1
    last_day = main_df['Day'].max()
    main_df['Week of month'] = main_df.apply(
    lambda row: Week_numtoimpute if row['Day'] <= 7-prev_hold_adj+1  else (row['Day']-(7-prev_hold_adj+1) -1)//7+adj_factor ,axis=1)
    remainder = (last_day-(7-prev_hold_adj)) % 7
   
    #print(last_day-remainder)
    if remainder<=3 and remainder!=0:
        last_week2impute=1
    elif remainder==0:
        last_week2impute=(last_day-(7-prev_hold_adj)) // 7+1
    else:
        last_week2impute=(last_day-(7-prev_hold_adj)) // 7+2

     
    main_df['Week of month'] = main_df.apply(
    lambda row: row['Week of month'] if row['Day'] <= last_day-remainder+1 else last_week2impute ,axis=1)
    
    max_week = main_df['Week of month'].max()
    prev_hold=remainder
    if remainder==0:
         prev_hold=7  
         max_week = main_df['Week of month'].max()-1
    main_df['rem']=remainder
    final_df = pd.concat([final_df, main_df], ignore_index=True)
        
#Month_year_mod=final_df.groupby(['Year', 'Month','Week of Year','']).agg({'Day':'count' }).reset_index()    
#Month_year_mod = Month_year_mod.sort_values(by='Day', ascending=True)
#Month_year_mod = Month_year_mod.drop_duplicates(subset=['Year', 'Month', 'Week of Year'], keep='first') 
#final_df = final_df.drop(columns=['Year', 'Month', 'column3'])
final_df.to_clipboard()


In [530]:
#Merge festival data with SKU demand data
unique_skus=sales_data_grouped_select['SKU_grouped'].unique()
final_df=final_df[['Date',"Year","Month","Week of Year",'Is Holiday','is_14_days_before_festival','is_a_week_before_holiday','is_7_days_before_festival',"Week of month",'Day','Week_Group']]
final_df['Date'] = pd.to_datetime(final_df['Date'])
sales_data_grouped_select['Date'] = pd.to_datetime(sales_data_grouped_select['Date'])
final_df1 = pd.DataFrame()
for sku in  unique_skus:
    main_df=sales_data_grouped_select[sales_data_grouped_select['SKU_grouped']==sku]
    start_date=min(main_df['Date'])
    end_date=max(main_df['Date'])
    #print(start_date)
    #print(end_date)
    SKU_df = final_df[(final_df['Date'] >= start_date) & (final_df['Date'] <= end_date)]
    SKU_df=SKU_df.merge(main_df,how='left',on='Date')
    SKU_df['SKU_grouped'] = SKU_df['SKU_grouped'].fillna(method='ffill') 
    SKU_df['Quantity'] = SKU_df['Quantity'].fillna(0) 
    final_df1=pd.concat([final_df1, SKU_df], ignore_index=True)
    
 #sales_data_grouped_select=   
#sales_data_grouped_select=sales_data_grouped_select.merge(date_map[["Date","Year","Week of Year",'Day of Year',"Week of Month","Is Weekend","Is Holiday"]],on='Date',how='left')
# sales_data_grouped_select['is_festival']=sales_data_grouped_select['is_festival'].fillna(0).astype(int)
print(sales_data_grouped_select.shape)
sales_data_grouped_select=final_df1.copy()
sales_data_grouped_select.to_clipboard()

# Group by 'Category' and 'SubCategory' and sum only 'Value1' and 'Value2'
result = sales_data_grouped_select.groupby(['Week of Year', 'Week of month','SKU_grouped','Week_Group']).agg({
    'Is Holiday': 'sum',                     # Sum column C
    'is_14_days_before_festival': 'sum', 
    'is_7_days_before_festival' : 'sum' ,                                 # Max value of column D
    'is_a_week_before_holiday': 'sum', 
    'Quantity': 'sum' ,
    'Day':'count',
    'Year': lambda x: x.mode()[0],
    'Month': lambda x: x.mode()[0] 
    }).reset_index()
#result1 = sales_data_grouped_select.groupby(['Week of Year', 'Week of month','SKU_grouped','Year','count']).agg({
#    'Day': 'count',
#    }).reset_index()
# Sorting by multiple columns
#result = result.sort_values(by=[ 'Year','Week of Year'], ascending=[True, True])
result.to_clipboard()

(6658, 7)


In [531]:
# Sorting by multiple columns
result['Week_sin']=np.sin(2*np.pi*result['Week of Year']/52)   
result['Week_cos']=np.cos(2*np.pi*result['Week of Year']/52)  
result=result.sort_values(by=[ 'Week_Group'], ascending=[True])
result['Lag_1']=result['Quantity'].shift(1)
result['Lag_2']=result['Quantity'].shift(2)
result['Lag_3']=result['Quantity'].shift(3)
result['Moving_Avg_3']=result['Quantity'].rolling(window=3).mean()
result['Moving_Avg_4']=result['Quantity'].rolling(window=4).mean()
result['Moving_std_3']=result['Quantity'].rolling(window=3).std()
result['Moving_std_4']=result['Quantity'].rolling(window=4).std()
result = result[result['Day'] >= 7]
result=result.dropna()
result1=result.copy()
#result.sort_values(by=[ 'Week_Group'], ascending=[True])
result.to_clipboard()

In [532]:
def wmape(y_true, y_pred):
    if np.sum(y_true) == 0:
        return 0
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_true)

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV


# %pip install scikit-learn
from sklearn.model_selection import ParameterGrid
from prophet import Prophet
# remove all warning messages
import warnings
warnings.filterwarnings("ignore")
# remove cmdstanpy warning for prophet
import logging
logger = logging.getLogger('cmdstanpy')
logger.propagate = False
logger.setLevel(logging.ERROR)

In [533]:
def xgb_forecast(train_df,test_df,forecasting_horizon,model_features):
    print("Inside xgb_Model")
    param_grid={"n_estimators":[25,75,50,100,150],
                "learning_rate":[0.01,0.05],
                "max_depth":[3,5,4,7]}
    train_data=train_df.copy()
    test_data=test_df.copy()

    Model_features=[str(col).replace(']','').replace('[','').replace('<','').replace('>','') for col in model_features]
    wmape_scorer=make_scorer(wmape,greater_is_better=False)
    model=XGBRegressor(objective='reg:squarederror')
    X_train=train_data[Model_features]
    y_train=train_data['Quantity']
    model.fit(X_train,y_train)

    # Use gridsearchCV for hyperparameter tuning
    grid_search=GridSearchCV(model,param_grid=param_grid,scoring=wmape_scorer,cv=5)
    grid_search.fit(X_train,y_train)

    # Get the best model
    best_model=grid_search.best_estimator_
    best_model.fit(X_train,y_train)
    train_predictions=best_model.predict(X_train)
    test_predictions=best_model.predict(test_data[Model_features])

    train_wmape=wmape(y_train,train_predictions)
    test_wmape=wmape(test_data['Quantity'],test_predictions)

    train_data['Prediction']=train_predictions
    test_data['Prediction']=test_predictions

    combined_data=pd.concat([train_data,test_data],axis=0)
    return combined_data[["SKU_grouped","Week_Group","Quantity","Prediction"]],train_wmape,test_wmape,best_model

params_grid={
            "seasonality_mode":("multiplicative","additive"),
            "changepoint_prior_scale":[0.1,0.3,0.5,0.7],
            "seasonality_prior_scale":[1.0,5.0,10.0],
            "n_changepoints":[25,35,50]

}
grid= ParameterGrid(params_grid)


def multivariate_prophet(train_df, test_df, forecasting_horizon, model_features):
    print("Inside multivariate_prophet")
    model_parameters=pd.DataFrame(columns=["WMAPE","Parameters"])
    train_data=train_df.copy()
    test_data=test_df.copy()
    train_data=train_data.rename(columns={"Date":"ds","Quantity":"y"})
    test_data=test_data.rename(columns={"Date":"ds","Quantity":"y"})
    Model_features=[str(col).replace(']','').replace('[','').replace('<','').replace('>','') for col in model_features]
    for p in grid:
        prophet_model=Prophet(changepoint_prior_scale = p["changepoint_prior_scale"],
                              n_changepoints=p["n_changepoints"],
                                seasonality_mode=p["seasonality_mode"],
                                seasonality_prior_scale=p["seasonality_prior_scale"])
        
        for reg in model_features:
            prophet_model.add_regressor(reg)

        prophet_model.add_country_holidays(country_name='IN')
        prophet_model.fit(train_data)
        future_dates=pd.date_range(start=train_df["Date"].max()+timedelta(days=1),periods=30,freq='D')
        future = pd.DataFrame({"ds": future_dates})
        for col in model_features:
            future[col] = test_df[col].values

        prophet_forecast=prophet_model.predict(future)
        test_predictions = prophet_forecast.loc[prophet_forecast['ds'].isin(test_df['Date'])]['yhat']

        WMAPE = wmape(test_df['Quantity'], test_predictions.values)

        x=pd.DataFrame({"WMAPE":WMAPE,"Parameters":[p]})
        model_parameters=pd.concat([model_parameters,x],ignore_index=False)

    params = model_parameters[model_parameters['WMAPE']==model_parameters['WMAPE'].min()]['Parameters'].values[0]

    model = Prophet(changepoint_prior_scale = params["changepoint_prior_scale"],
                    n_changepoints=params["n_changepoints"],
                    seasonality_mode=params["seasonality_mode"],
                    seasonality_prior_scale=params["seasonality_prior_scale"],
                    interval_width=0.95)
    for col in model_features:
        model.add_regressor(col)

    model.fit(train_data)

    future = model.make_future_dataframe(periods=30, freq='D')
    for col in model_features:
        future[col] = pd.concat([train_df,test_df])[col].values
    forecast = model.predict(future)
    train_wmape = wmape(train_data['y'], model.predict(train_data)['yhat'])
    test_wmape = wmape(test_data['y'], forecast["yhat"].tail(forecasting_horizon))
    train_data['Prediction'] = model.predict(train_data)['yhat'].values
    train_data["Set"]="Train"

    test_data['Prediction']=forecast["yhat"].tail(forecasting_horizon).values
    test_data["Set"]="Test"

    combined_data=pd.concat([train_data,test_data],axis=0)
    combined_data['SKU_grouped']=sku
    combined_data.rename(columns={"ds":"Date","y":"Quantity"},inplace=True)
    return combined_data[["SKU_grouped","Date","Quantity","Prediction"]],train_wmape,test_wmape,model

def multi_prophet_forecasting(main_df, test_df, prophet_model, model_features, forecasting_horizon=60):
    print("Inside multi_prophet_forecasting")
    future_dates=pd.date_range(start=test_df["Date"].max()+timedelta(days=1),periods=forecasting_horizon,freq='D')
    future=pd.DataFrame({"ds":future_dates})
    future['Year'] = future['ds'].dt.year                       # Extract year
    future['Week of Year'] = future['ds'].dt.isocalendar().week  # ISO week number
    # add week of month also
    future['Week of Month'] = (future['ds'].dt.day-1) // 7 + 1
    # Add Is Weekend flag
    future['Day of Week'] = future['ds'].dt.day_name()          # Day name (e.g., Sunday, Monday)

    # add day of Year
    future['Day of Year'] = future['ds'].dt.dayofyear
    future['Week Ending Date'] = future['ds'] + pd.offsets.Week(weekday=5)  # Saturday as week-ending date
    # Add holiday names
    future['Holiday Name'] = future['ds'].dt.strftime('%Y-%m-%d').map(indian_holidays)
    # Add a column to mark whether the date is a holiday or weekend
    future['Is Weekend'] = np.where(future['Day of Week'].isin(['Saturday', 'Sunday']),1,0)
    future['Is Holiday'] = future['Holiday Name'].notna().astype(int)
    future['is_14_days_before_holiday']=0
    for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
        future.loc[
            (future['ds']>=date - pd.Timedelta(days=14)) & (future['ds']<date),
            'is_14_days_before_festival']=1
    future['is_7_days_before_holiday']=0
    for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
        future.loc[
            (future['ds']>=date-pd.Timedelta(days=7)) & (future['ds']<date),
            'is_7_days_before_festival']=1
    date_map
    # add demand lag 7 by taking values from last year and merging it by week of year
    future=future.merge(main_df[['Day of Year','demand_lag_7','demand_lag_1','demand_lag_14','rolling mean 7','rolling std 7']].drop_duplicates(),on=['Day of Year'],how='left')
    future['day of week']=future['ds'].dt.dayofweek
    future['day of month']=future['ds'].dt.month
    future.fillna(0,inplace=True)
    future['Week of Year'] = future['Week of Year'].astype(int)
    future['Prediction']=prophet_model.predict(future)["yhat"]
    future["Quantity"]=np.nan
    future['SKU_grouped']=sku
    future['Date']=future['ds']
    return future[["SKU_grouped","Date","Prediction"]]


def xgb_forecaster(main_df,test_df, xgb_model, model_features, forecasting_horizon=1):   
    print("Inside xgb_forecaster") 
    start=test_df["Week_Group"].max()+1
    future=pd.DataFrame({"Date":future_dates})

    future['Year'] = future['Date'].dt.year                       # Extract year
    future['Week of Year'] = future['Date'].dt.isocalendar().week  # ISO week number
    # add week of month also
    future['Week of Month'] = (future['Date'].dt.day-1) // 7 + 1
    # Add Is Weekend flag
    future['Day of Week'] = future['Date'].dt.day_name()          # Day name (e.g., Sunday, Monday)
    # add day of Year
    future['Day of Year'] = future['Date'].dt.dayofyear
    future['Week Ending Date'] = future['Date'] + pd.offsets.Week(weekday=5)  # Saturday as week-ending date
    # Add holiday names
    future['Holiday Name'] = future['Date'].dt.strftime('%Y-%m-%d').map(indian_holidays)
    # Add a column to mark whether the date is a holiday or weekend
    future['Is Weekend'] = np.where(future['Day of Week'].isin(['Saturday', 'Sunday']),1,0)
    future['Is Holiday'] = future['Holiday Name'].notna().astype(int)
    future['is_14_days_before_holiday']=0
    for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
        future.loc[
            (future['Date']>=date - pd.Timedelta(days=14)) & (future['Date']<date),
            'is_14_days_before_festival']=1
    future['is_7_days_before_holiday']=0
    for date in date_map[date_map['Is Holiday']==1]["Date"].unique():
        future.loc[
            (future['Date']>=date-pd.Timedelta(days=7)) & (future['Date']<date),
            'is_7_days_before_festival']=1

    # add demand lag 7 by taking values from last year and merging it by week of year
    future=future.merge(main_df[['Day of Year','demand_lag_7','demand_lag_1','demand_lag_14','rolling mean 7','rolling std 7']].drop_duplicates(),on=['Day of Year'],how='left')


    future['day of week']=future['Date'].dt.dayofweek
    future['day of month']=future['Date'].dt.month
    future.fillna(0,inplace=True)
    future['Prediction']=xgb_model.predict(future[model_features])
    future["Quantity"]=np.nan
    future['SKU_grouped']=sku
    return future
    #return future[["SKU_grouped","Date","Prediction"]]


In [ ]:
forecast_df=pd.DataFrame()

results=pd.DataFrame(columns=['SKU','Model','Train WMAPE','Test WMAPE'])
for sku in unique_skus:
    print(f'Forecasting for SKU:{sku}')
    
    main_df=result1[result1['SKU_grouped']==sku]
    
    max_date=main_df['Week_Group'].max()
    max_train_date=main_df['Week_Group'].max()-3
    main_df['Week of Year'] = main_df['Week of Year'].astype(int)
    train_df=main_df[main_df['Week_Group']<=max_train_date]
    test_df=main_df[main_df['Week_Group']>max_train_date]

    model_features=["Year",'Week of Year', 'Week of month', 'Is Holiday', 'is_14_days_before_festival', 'is_7_days_before_festival', 'is_a_week_before_holiday',
       'Month', 'Week_sin', 'Week_cos', 'Lag_1','Lag_2','Lag_3',	'Moving_Avg_3','Moving_Avg_4','Moving_std_3','Moving_std_4']
    combined_data_xgb,train_wmape,test_wmape,xgb_model=xgb_forecast(train_df,test_df,3,model_features)
    future=xgb_forecaster(main_df,test_df,xgb_model,model_features,1)
    results.loc[len(results)] = [sku, "XGBoost", train_wmape, test_wmape]
    #sku_df_xgb=combined_data_xgb.copy()
    sku_df_xgb=pd.concat([combined_data_xgb,future])
    sku_df_xgb["Train_wmape"]=train_wmape
    sku_df_xgb["Test_wmape"]=test_wmape
    sku_df_xgb["Model"]="XGBoost"
    print(f'WMAPE for {sku}:{train_wmape} (train) and {test_wmape} (test)')
    #combined_data_prophet,train_wmape,test_wmape,prophet_model=multivariate_prophet(train_df,test_df,30,model_features)
    #future=multi_prophet_forecasting(main_df,test_df,prophet_model,model_features,60)
    #results.loc[len(results)] = [sku, "Prophet", train_wmape, test_wmape]
    #sku_df=pd.concat([combined_data_prophet,future])
    #sku_df["Train_wmape"]=train_wmape
    #sku_df["Test_wmape"]=test_wmape
    #sku_df["Model"]="Prophet"
    #print(f'WMAPE for {sku}:{train_wmape} (train) and {test_wmape} (test)')
    forecast_df=pd.concat([forecast_df,sku_df_xgb],axis=0)
    #forecast_df=pd.concat([forecast_df,sku_df],axis=0)


Forecasting for SKU:MuscleBlaze Biozyme Performance Whey
Inside xgb_Model
WMAPE for MuscleBlaze Biozyme Performance Whey:0.0025883899468606324 (train) and 0.09848528764745053 (test)
Forecasting for SKU:MuscleBlaze Chocolate Peanut Butter
Inside xgb_Model
WMAPE for MuscleBlaze Chocolate Peanut Butter:0.01218687961249024 (train) and 0.5223667287380895 (test)
Forecasting for SKU:MuscleBlaze 80% Raw Whey Protein Supplement Powder
Inside xgb_Model
WMAPE for MuscleBlaze 80% Raw Whey Protein Supplement Powder:0.17234792535955257 (train) and 0.6270174425701762 (test)
Forecasting for SKU:MuscleBlaze MB-VITE Daily Multivitamin
Inside xgb_Model
WMAPE for MuscleBlaze MB-VITE Daily Multivitamin:0.1527226126565474 (train) and 0.391455534732703 (test)
Forecasting for SKU:MB Fuel One Whey Protein
Inside xgb_Model
WMAPE for MB Fuel One Whey Protein:0.24403741948173854 (train) and 0.3712436252170139 (test)
Forecasting for SKU:MuscleBlaze High Protein Muesli
Inside xgb_Model
WMAPE for MuscleBlaze High Pr

In [536]:
forecast_df.to_clipboard() 